In [ ]:
using Pkg
using Revise
Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
using GLMakie
# Constants and Parameters
const n, a, b = 3, 6, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = old_alpine1 # Objective function

  Activating project at `~/globtim`


#14 (generic function with 1 method)

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased. For `a/b= 6`, we capture the right structure at `d=14`, but the $L^2$-norm is still quite high (around 1.0 ). 

In [27]:
rand_center = [0.0, 0.0, 0.];
d = 16 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev);
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.7118973975570098
current L2-norm: 0.6887491987834493


Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [29]:
# df_cheb = solve_and_parse(pol_cheb, x, f, TR)
pts_cheb = solve_polynomial_system(x, TR.dim, d, pol_cheb.coeffs; basis=:chebyshev)
df_cheb = process_critical_points(pts_cheb, f, TR)
sort!(df_cheb, :z, rev=false)

Dimension m of the vector space: 969
Tracking 3375 paths...   0%|                            |  ETA: 2:33:40



Tracking 3375 paths...   1%|▍                           |  ETA: 0:06:50



Tracking 3375 paths...   3%|▊                           |  ETA: 0:03:30



Tracking 3375 paths...   4%|█▏                          |  ETA: 0:02:31



Tracking 3375 paths...   5%|█▍                          |  ETA: 0:01:57



Tracking 3375 paths...   6%|█▊                          |  ETA: 0:01:36



Tracking 3375 paths...   8%|██▏                         |  ETA: 0:01:20



Tracking 3375 paths...   9%|██▌                         |  ETA: 0:01:10



Tracking 3375 paths...  10%|██▉                         |  ETA: 0:01:03



Tracking 3375 paths...  11%|███▏                        |  ETA: 0:00:58



Tracking 3375 paths...  12%|███▌                        |  ETA: 0:00:53



Tracking 3375 paths...  14%|███▉                        |  ETA: 0:00:49



Tracking 3375 paths...  15%|████▏                       |  ETA:

Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,-0.0342924,-0.0342924,-0.0342924,0.0067605
2,-0.0342924,-0.0342924,-3.00992,0.0986928
3,-3.00992,-0.0342924,-0.0342924,0.0986928
4,-0.0342924,-3.00992,-0.0342924,0.0986928
5,-0.0342924,-0.0342924,3.2123,0.0987959
6,-0.0342924,3.2123,-0.0342924,0.0987959
7,3.2123,-0.0342924,-0.0342924,0.0987959
8,-3.00992,-3.00992,-0.0342924,0.190625
9,-3.00992,-0.0342924,-3.00992,0.190625


In [25]:
grid = scale_factor * generate_grid(3, 100)  # 3D grid
values = map(f, grid); # Prepare level set data for specific level

In [30]:
fig = create_level_set_visualization(f, grid, df_cheb, (0., 16.))

Interesting: Why is Legendre approximant so much slower than Chebyshev approximant for computing the critical locus ? 
- []: There is a sparsity difference on the support 
- []: There is a difference in the magnitudes of the coefficients
- []: There are more critical points captured by Legendre approximant? We should have the same number of complex solutions in both cases ? 

In [31]:
pts_lege = solve_polynomial_system(x, TR.dim, d, pol_lege.coeffs; basis=:legendre)
df_lege = process_critical_points(pts_lege, f, TR)
sort!(df_lege, :z, rev=false)

Dimension m of the vector space: 969
Tracking 3375 paths...   0%|                            |  ETA: 0:16:19



Tracking 3375 paths...   0%|                            |  ETA: 0:17:21



Tracking 3375 paths...   0%|                            |  ETA: 0:21:57



Tracking 3375 paths...   0%|                            |  ETA: 0:22:43



Tracking 3375 paths...   0%|                            |  ETA: 0:22:22



Tracking 3375 paths...   0%|                            |  ETA: 0:22:01



Tracking 3375 paths...   0%|▏                           |  ETA: 0:21:35



Tracking 3375 paths...   0%|▏                           |  ETA: 0:21:03



Tracking 3375 paths...   0%|▏                           |  ETA: 0:20:49



Tracking 3375 paths...   0%|▏                           |  ETA: 0:20:57



Tracking 3375 paths...   0%|▏                           |  ETA: 0:21:25



Tracking 3375 paths...   0%|▏                           |  ETA: 0:21:14



Tracking 3375 paths...   0%|▏                           |  ETA:

Excessive output truncated after 524308 bytes.

Tracking 3375 paths...  34%|█████████▌                  |  ETA: 0:18:55


Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,-0.0483021,-0.0483021,-0.0483021,0.00749407
2,-0.0483021,-3.01885,-0.0483021,0.0727329
3,-3.01885,-0.0483021,-0.0483021,0.0727329
4,-0.0483021,-0.0483021,-3.01885,0.0727329
5,-0.0483021,-0.0483021,3.21893,0.0782018
6,-0.0483021,3.21893,-0.0483021,0.0782018
7,3.21893,-0.0483021,-0.0483021,0.0782018
8,-3.01885,-3.01885,-0.0483021,0.137972
9,-0.0483021,-3.01885,-3.01885,0.137972


In [ ]:
fig = create_level_set_visualization(f, grid, df_lege, (0.0, 16.0))

In [32]:
GLMakie.closeall()